<a href="https://colab.research.google.com/github/k-tsymbaliuk/python-eda-project/blob/main/notebooks/apartments_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages (for Google Colab or fresh environments)
!pip install ydata-profiling

In [ ]:
# In Google Colab, data was loaded from Google Drive.
# For this repository, data is expected to be located in the /data folder.

In [ ]:
import pandas as pd
import ydata_profiling

In [ ]:
appartments_df = pd.read_csv("data/appartments_data_set.csv")
appartments_df.head(5)

In [ ]:
#profiler = ydata_profiling.ProfileReport(appartments_df)
#profiler.to_file("data/appartments_data_set.csv")

1. Download data from the appartments_data_set.csv file. This dataset contains descriptions of apartments listed for sale.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from scipy.signal import find_peaks
from IPython.display import display

2. Perform some initial data processing. Add a column for the cost per square meter of housing.

In [ ]:
appartments_df = appartments_df.drop(columns=['is_apartment', 'days_exposition'], errors='ignore')

appartments_df['locality_name_clean'] = appartments_df['locality_name'].astype(str).str.strip().str.lower()

for col in ['total_area', 'living_area', 'kitchen_area']:
    appartments_df[col] = appartments_df[col].replace(0, np.nan)

appartments_df['first_day_exposition'] = pd.to_datetime(appartments_df['first_day_exposition'], errors='coerce')

appartments_df['price_per_m2'] = appartments_df['last_price'] / appartments_df['total_area']

display(appartments_df[['last_price', 'total_area', 'price_per_m2']].head())

3. Perform exploratory data analysis.

In [ ]:
print("\nNumerical Column Statistics:")
display(appartments_df.describe())

print("\nNumber of skips by columns:")
display(appartments_df.isna().sum())

plt.hist(appartments_df['price_per_m2'].dropna(), bins=50, color='skyblue')
plt.xlabel('Price per m²')
plt.ylabel('Number of apartments')
plt.title('Price distribution per m²')
plt.show()

4. What is the number of rooms in apartments most often sold in Kyiv?

In [ ]:
kyiv_df = appartments_df[appartments_df['locality_name_clean'] == 'kyiv']

if not kyiv_df.empty:
    rooms_mode = kyiv_df['rooms'].mode()
    if not rooms_mode.empty:
        kyiv_rooms_mode = rooms_mode[0]
        print(f"in Kyiv, apartments with {kyiv_rooms_mode} rooms are most often sold")
    else:
        print("failed to determine the number of rooms in Kyiv")

    # Number of apartments for each number of rooms
    rooms_count = kyiv_df['rooms'].value_counts().sort_index()
    print("\nnumber of apartments sold by number of rooms in Kyiv:")
    display(rooms_count)

else:
    print("No sold apartments in Kyiv were found in the dataset")

5. The average cost of a one-room apartment in Irpen?

In [ ]:
irpin_df = appartments_df[appartments_df['locality_name_clean'] == 'irpin']
irpin_1room_avg = irpin_df[irpin_df['rooms'] == 1]['last_price'].mean()
print(f"average price of a 1-room apartment in Irpin: {irpin_1room_avg:,.0f} UAH")

6. Are studio apartments popular?

In [ ]:
total_apartments = appartments_df.shape[0]

studio_count = appartments_df['studio'].sum()

studio_share = (studio_count / total_apartments * 100) if total_apartments > 0 else 0

popularity = "popular" if studio_share >= 10 else "not very popular"

print(f"total number of apartments sold: {total_apartments}")
print(f"number of studio apartments sold: {studio_count}")
print(f"percentage of studio apartments sold out of all apartments: {studio_share:.2f}%")
print(f"Conclusion: sale of studio apartments {popularity}")

7. State a hypothesis about the existence of a multimodal distribution of the cost of a square meter in Kyiv. What can it be caused by?

In [ ]:
kyiv_prices = kyiv_df['price_per_m2'].dropna()

plt.hist(kyiv_prices, bins=30, density=True, alpha=0.5, color='skyblue', label='Гістограма')

kde = gaussian_kde(kyiv_prices)
x = np.linspace(kyiv_prices.min(), kyiv_prices.max(), 1000)
plt.plot(x, kde(x), color='red', label='KDE')

plt.xlabel('Price per m²')
plt.ylabel('Density')
plt.title('Price distribution per m² in Kyiv (histogram + KDE)')
plt.legend()
plt.show()

peaks, _ = find_peaks(kde(x))
print(f"Number of peaks (sign of multimodality): {len(peaks)}")

print("\nHypothesis:")
print("The multimodal distribution of prices per m² in Kyiv can be associated with different districts (center/suburbs), ")
print("housing types (new buildings/resale), and building classes (economy/business/elite).")

**CEILING HEIGHT**

In [ ]:
print("Minimum height:", appartments_df['ceiling_height'].min(skipna=True))
print("Maximum height:", appartments_df['ceiling_height'].max(skipna=True))

In [ ]:
bad_heights = appartments_df[(appartments_df['ceiling_height'] < 2) |
                             (appartments_df['ceiling_height'] > 6)]

print("number of apartments with unrealistic height:", len(bad_heights))

display(bad_heights[['ceiling_height','total_area','rooms','locality_name','last_price']])

**TASK to identify all discrepancies!**

In [ ]:
rules = {
    'total_area': (10, 500),        # area 10–500 m²
    'living_area': (5, 400),        # living area 5–400 m²
    'kitchen_area': (2, 150),       # kitchen 2–150 m²
    'balcony': (0, 50),             # balcony 0–50 m²
    'ceiling_height': (2, 6),       # ceiling 2–6 m
    'rooms': (0, 20),               # rooms 0–20
    'floor': (0, 50),               # floor 0–50
    'floors_total': (1, 50),        # floors in the building 1–50
    'last_price': (10000, 2e8),     # price 10 thousand – 200 million
}

if 'price_per_m2' not in appartments_df.columns:
    appartments_df['price_per_m2'] = appartments_df['last_price'] / appartments_df['total_area']
rules['price_per_m2'] = (5000, 150000)

problems_list = []

for col, (min_val, max_val) in rules.items():
    if col in appartments_df.columns:
        too_low = appartments_df[col] < min_val
        too_high = appartments_df[col] > max_val
        for idx in appartments_df[too_low].index:
            problems_list.append((idx, col, f"{col} < {min_val}", appartments_df.loc[idx, col]))
        for idx in appartments_df[too_high].index:
            problems_list.append((idx, col, f"{col} > {max_val}", appartments_df.loc[idx, col]))

for idx, row in appartments_df.iterrows():
    if row['living_area'] > row['total_area']:
        problems_list.append((idx, 'living_area', 'living_area > total_area', row['living_area']))
    if row['kitchen_area'] > row['total_area']:
        problems_list.append((idx, 'kitchen_area', 'kitchen_area > total_area', row['kitchen_area']))
    if row['floor'] > row['floors_total']:
        problems_list.append((idx, 'floor', 'floor > floors_total', row['floor']))
    if row['total_area'] < (row['living_area'] + row['kitchen_area'] - 5):
        problems_list.append((idx, 'total_area', 'total_area < living_area + kitchen_area - 5', row['total_area']))

bad_data = pd.DataFrame(problems_list, columns=['row_index','column','problem_type','value'])
bad_data = bad_data.merge(appartments_df, left_on='row_index', right_index=True)

print("problems found:", len(bad_data))
display(bad_data[['row_index','column','problem_type','value',
                  'last_price','total_area','living_area','kitchen_area',
                  'balcony','ceiling_height','rooms','floor','floors_total',
                  'price_per_m2','locality_name']])

In [ ]:
bad_data.to_excel("bad_data.xlsx", index=False)
from google.colab import files
files.download("bad_data.xlsx")

In [ ]:
bad_data.to_csv("bad_data.csv", index=False)

# Download the file to your computer
from google.colab import files
files.download("bad_data.csv")

In [ ]:
problems_list = []

for idx, row in appartments_df.iterrows():
    # 1️⃣ The ceiling height is unrealistic.
    if row['ceiling_height'] < 2 or row['ceiling_height'] > 6:
        problems_list.append((idx, 'ceiling_height', row['ceiling_height']))

    # 2️⃣ The apartment is on a higher floor than the number of floors in the house.
    if pd.notna(row['floors_total']) and row['floor'] > row['floors_total']:
        problems_list.append((idx, 'floor', row['floor']))

    # 3️⃣ Incorrect area ratio
    if pd.notna(row['living_area']) and pd.notna(row['kitchen_area']) and pd.notna(row['total_area']):
        if row['total_area'] < (row['living_area'] + row['kitchen_area'] - 5):
            problems_list.append((idx, 'area_relation', row['total_area']))

    # 4️⃣ Too large balcony area
    if pd.notna(row['balcony']) and row['balcony'] > 30:
        problems_list.append((idx, 'balcony', row['balcony']))

bad_data = pd.DataFrame(problems_list, columns=['row_index', 'problem_type', 'value'])

In [ ]:
# Get unique indexes of problem rows
bad_indexes = bad_data['row_index'].unique()

# Delete these rows from the main dataframe
clean_df = appartments_df.drop(index=bad_indexes)

print(f"There were lines: {len(appartments_df)}")
print(f"Problematic lines removed: {len(bad_indexes)}")
print(f"Left over after cleaning: {len(clean_df)}")

In [ ]:
print("Minimum height:", clean_df['ceiling_height'].min(skipna=True))
print("Maximum height:", clean_df['ceiling_height'].max(skipna=True))
print("Maximum balcony area:", clean_df['balcony'].max(skipna=True))

In [ ]:
clean_df.to_csv("appartments_data_set_clean.csv", index=False)

# Download to your computer
from google.colab import files
files.download("appartments_data_set_clean.csv")